In [2]:
import sys

sys.path.append("..")

%load_ext autoreload
%autoreload complete

In [13]:
from math import ceil
from pathlib import Path

import dask.array
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr

from project.plotting import plot_field, format_plot, save_plot
from project.util import stack_state, unstack_state, TimeConverter
from project.spaces import PhysicalSpaceForecastSpaceMapper
from project.lim import LIM
from project.io import save_mfdataset
from project.kf import SerialEnSRF, create_initial_ensemble_from_sample

In [4]:
data_true = xr.open_mfdataset(
    Path("/home/enkf6/dstiller/enspred/annual_averages/2023-12-08T00-58-19").glob("**/*.nc")
)
data_true = data_true.assign_coords(time=range(1, len(data_true.time) + 1))
data_true

OSError: no files to open

In [ ]:
obs = xr.open_mfdataset(
    Path("/home/enkf6/dstiller/enspred/obs/2023-12-09T21-21-21").glob("**/*.nc")
).set_xindex(["lon", "lat"])
obs

<xarray.Dataset>
Dimensions:   (time: 999, location: 364)
Coordinates:
  * time      (time) int64 850 851 852 853 854 855 ... 1844 1845 1846 1847 1848
  * location  (location) object MultiIndex
  * lon       (location) float64 0.0 2.0 4.0 6.0 6.0 ... 356.0 356.0 356.0 358.0
  * lat       (location) float64 43.0 43.0 41.0 -75.0 ... 35.0 37.0 43.0 -75.0
Data variables:
    tas       (time, location) float32 dask.array<chunksize=(100, 364), meta=np.ndarray>

In [16]:
create_initial_ensemble_from_perturbations(
    xr.open_mfdataset(
        Path("/home/enkf6/dstiller/enspred/annual_averages/2023-12-09T23-33-06").glob("**/*.nc")
    ).isel(time=542),
    10,
)

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 90, ens: 10)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
    time     int64 543
  * ens      (ens) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    ohc700   (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    pr       (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    psl      (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    rlut     (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    rsut     (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    sos      (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    tas      (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    tos      (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    zg500    (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>
    zos      (ens, lon, lat) float64 dask.array<chunksize=(1, 180, 90), meta=np.ndarray>

In [17]:
create_initial_ensemble_from_sample(
    xr.open_mfdataset(
        Path("/home/enkf6/dstiller/enspred/annual_averages/2023-12-09T23-33-06").glob("**/*.nc")
    ),
    10,
    850,
)

2023-12-12 14:25:49    INFO Drawing IC from years [1804  452 1699 1386 1131   55  327 1211  130 1015]


<xarray.Dataset>
Dimensions:  (lon: 180, lat: 90, ens: 10)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * ens      (ens) int64 0 1 2 3 4 5 6 7 8 9
    time     int64 850
Data variables:
    ohc700   (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    pr       (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    psl      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rlut     (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rsut     (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    sos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tas      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zg500    (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>

In [12]:
initial = xr.open_mfdataset(
    Path("/home/enkf6/dstiller/enspred/annual_averages/2023-12-09T23-33-06").glob("**/*.nc")
)
initial = (
    initial.isel(
        time=np.random.default_rng(682652).choice(np.arange(len(initial.time)), 20, replace=False)
    )
    .rename(time="ens")
    .assign_coords(ens=range(20))
)
initial

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 90, ens: 20)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * ens      (ens) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
Data variables:
    ohc700   (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    pr       (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    psl      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rlut     (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rsut     (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    sos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tas      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zg500    (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zos      (ens, lat, lon) float32 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>

In [ ]:
mapper = PhysicalSpaceForecastSpaceMapper.load(
    Path("/home/enkf6/dstiller/enspred/mapper/mapper-2023-12-08T00-12-39.pkl")
)

In [ ]:
lim = LIM.load(Path("/home/enkf6/dstiller/enspred/lim/lim-2023-12-09T20-20-50.pkl"))

In [ ]:
initial_state = xr.DataArray(
    mapper.forward(initial), coords=dict(state=range(50), time=initial.time)
)
fc_state = lim.forecast_stochastic(initial_state, 1, 10, 1440, initial.time.item()).isel(
    time=slice(-1)
)
fc_physical = mapper.backward_ensemble(fc_state)
fc_physical

2023-12-09 23:32:16    INFO PhysicalSpaceForecastSpaceMapper.forward()
2023-12-09 23:32:16    INFO Splitting dataset into Dask arrays


2023-12-09 23:32:16    INFO Masking nans
2023-12-09 23:32:16    INFO Detrending data
2023-12-09 23:32:16    INFO Standardizing pr before individual EOF
2023-12-09 23:32:16    INFO Projecting EOF for ohc700 [1/10]
2023-12-09 23:32:16    INFO Standardizing ohc700 after individual EOF
2023-12-09 23:32:16    INFO Projecting EOF for pr [2/10]
2023-12-09 23:32:16    INFO Projecting EOF for psl [3/10]
2023-12-09 23:32:16    INFO Standardizing psl after individual EOF
2023-12-09 23:32:16    INFO Projecting EOF for rlut [4/10]
2023-12-09 23:32:16    INFO Standardizing rlut after individual EOF
2023-12-09 23:32:17    INFO Projecting EOF for rsut [5/10]
2023-12-09 23:32:17    INFO Standardizing rsut after individual EOF
2023-12-09 23:32:17    INFO Projecting EOF for sos [6/10]
2023-12-09 23:32:17    INFO Standardizing sos after individual EOF
2023-12-09 23:32:17    INFO Projecting EOF for tas [7/10]
2023-12-09 23:32:17    INFO Standardizing tas after individual EOF
2023-12-09 23:32:17    INFO Pro

<xarray.Dataset>
Dimensions:  (lon: 180, lat: 90, ens: 10)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
    time     float64 850.0
  * ens      (ens) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    ohc700   (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    pr       (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    psl      (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rlut     (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    rsut     (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    sos      (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tas      (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    tos      (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zg500    (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>
    zos      (ens, lat, lon) float64 dask.array<chunksize=(1, 90, 180), meta=np.ndarray>

In [ ]:
prio = (
    xr.open_mfdataset(
        Path("/home/enkf6/dstiller/enspred/annual_averages/2023-12-09T23-33-06").glob("**/*.nc")
    )
    .isel(time=0)
    .expand_dims("time")
)
mapper.forward_ensemble(create_initial_ensemble(prio, 10))

2023-12-10 00:16:46    INFO PhysicalSpaceForecastSpaceMapper.forward()
2023-12-10 00:16:46    INFO Splitting dataset into Dask arrays
2023-12-10 00:16:46    INFO Masking nans
2023-12-10 00:16:46    INFO Detrending data
2023-12-10 00:16:46    INFO Standardizing pr before individual EOF
2023-12-10 00:16:46    INFO Projecting EOF for ohc700 [1/10]
2023-12-10 00:16:46    INFO Standardizing ohc700 after individual EOF
2023-12-10 00:16:46    INFO Projecting EOF for pr [2/10]
2023-12-10 00:16:46    INFO Projecting EOF for psl [3/10]
2023-12-10 00:16:46    INFO Standardizing psl after individual EOF
2023-12-10 00:16:46    INFO Projecting EOF for rlut [4/10]
2023-12-10 00:16:46    INFO Standardizing rlut after individual EOF
2023-12-10 00:16:46    INFO Projecting EOF for rsut [5/10]
2023-12-10 00:16:47    INFO Standardizing rsut after individual EOF
2023-12-10 00:16:47    INFO Projecting EOF for sos [6/10]
2023-12-10 00:16:47    INFO Standardizing sos after individual EOF
2023-12-10 00:16:47   

<xarray.Dataset>
Dimensions:                                       (state: 50, time: 1, ens: 10)
Coordinates:
  * state                                         (state) int64 0 1 2 ... 48 49
  * time                                          (time) int64 1
  * ens                                           (ens) int64 0 1 2 3 ... 7 8 9
Data variables:
    concatenate-2c7806095883873d68cba298d01f51a7  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-46358cd0fcbf797f59940475e7a6ff39  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-5a9d87d484c5728318bd86b5191db798  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-70ad808fda987de32e55e058853a6432  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-8b3d703618cc492f2ff48abec585661a  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-9fa90ffcf8fa3c9d0305e46ba8f9d2c5  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-bc95d0db0cf51ff1bd502e3409eb8e76  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-cd49483d29d9069fc6f1fe364cddbb75  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-dfd8be2ca30d561636161714748c51e3  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>
    concatenate-e41875da7a20d875864ada5c2cb6b696  (ens, state, time) float64 dask.array<chunksize=(10, 30, 1), meta=np.ndarray>

In [ ]:
assim = SerialEnSRF().assimilate(
    fc_physical, obs.sel(time=fc_physical.time).isel(location=slice(3)), 1
)
assim

/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 10
  out = blockwise(
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 10
  out = blockwise(
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 10
  out = blockwise(
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 10
  out = blockwise(
/home/disk/p/dstiller/.local/software/miniconda3/envs/enspred/lib/python3.11/site-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 10
  out = b

<xarray.Dataset>
Dimensions:  (time: 1, ens: 10, lon: 180, lat: 90)
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * ens      (ens) int64 0 1 2 3 4 5 6 7 8 9
  * time     (time) float64 850.0
Data variables:
    ohc700   (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    pr       (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    psl      (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    rlut     (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    rsut     (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    sos      (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    tas      (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    tos      (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    zg500    (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>
    zos      (time, ens, lon, lat) float64 dask.array<chunksize=(1, 1, 180, 90), meta=np.ndarray>